# Import Tools

In [1]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import torch.nn.functional as F
torch.manual_seed(42)
np.random.seed(42)
from torch_geometric.nn import GATConv
from torch_geometric.data import Data
from torch_geometric.nn import knn_graph
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.feature_selection import VarianceThreshold
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

C:\Users\MEOW\AppData\Roaming\Python\Python312\site-packages\torch_geometric\typing.py:68: UserWarning: An issue occurred while importing 'pyg-lib'. Disabling its usage. Stacktrace: [WinError 127] The specified procedure could not be found
  warnings.warn(f"An issue occurred while importing 'pyg-lib'. "
C:\Users\MEOW\AppData\Roaming\Python\Python312\site-packages\torch_geometric\typing.py:86: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: [WinError 127] The specified procedure could not be found
  warnings.warn(f"An issue occurred while importing 'torch-scatter'. "
C:\Users\MEOW\AppData\Roaming\Python\Python312\site-packages\torch_geometric\typing.py:97: UserWarning: An issue occurred while importing 'torch-cluster'. Disabling its usage. Stacktrace: [WinError 127] The specified procedure could not be found
  warnings.warn(f"An issue occurred while importing 'torch-cluster'. "
C:\Users\MEOW\AppData\Roaming\Python\Python312\site-packages\

# Data load

In [2]:
expression_data = pd.read_csv('TCGA-SARC.star_tpm.tsv', sep='\t', index_col=0)  # Gene expression (TPM)
methylation_data = pd.read_csv('TCGA-SARC.methylation450.tsv', sep='\t', index_col=0)  # DNA methylation
copy_number_data = pd.read_csv('TCGA-SARC.gene-level_absolute.tsv', sep='\t', index_col=0)  # Gene-level copy number (ABSOLUTE)
protein_data = pd.read_csv('TCGA-SARC.protein.tsv', sep='\t', index_col=0)  # Protein expression

try:
    phenotype_data = pd.read_csv('TCGA-SARC.clinical.tsv', sep='\t', index_col=0)
except Exception as e:
    print("Error while loading phenotype_data:", e)
    with open('TCGA-SARC.clinical.tsv', 'r') as f:
        for i, line in enumerate(f):
            try:
                pd.read_csv(pd.compat.StringIO(line), sep='\t')
            except Exception as row_e:
                print(f"Error in line {i}: {row_e}")
    phenotype_data = pd.read_csv('TCGA-SARC.clinical.tsv', sep='\t', index_col=0, on_bad_lines='skip')

print("Data shapes:")
print(f"Expression data: {expression_data.shape}")
print(f"Methylation data: {methylation_data.shape}")
print(f"Copy number data: {copy_number_data.shape}")
print(f"Protein data: {protein_data.shape}")
print(f"Phenotype data: {phenotype_data.shape}")

FileNotFoundError: [Errno 2] No such file or directory: 'TCGA-SARC.star_tpm.tsv'

# Sample Matching

In [ ]:
# subtype_columns = [col for col in phenotype_data.columns if any(keyword in col.lower() 
#                    for keyword in ['subtype', 'histology', 'type', 'classification'])]

# print("Subtype-related columns:")
# for col in subtype_columns:
#     print(f"- {col}")
#     print(phenotype_data[col].value_counts())
#     print()


print("Checking for missing values in phenotype data:")
missing_values = phenotype_data.isnull().sum()
print(missing_values[missing_values > 0])

print()
print("Checking sample overlap between different omics data:")

samples_expression = set(expression_data.columns)
samples_methylation = set(methylation_data.columns)
samples_cnv = set(copy_number_data.columns)
samples_clinical = set(phenotype_data.index)
samples_protein = set(protein_data.columns)
print("Sample overlap:")
print(f"Expression samples: {len(samples_expression)}")
print(f"Methylation samples: {len(samples_methylation)}")
print(f"CNV samples: {len(samples_cnv)}")
print(f"Clinical samples: {len(samples_clinical)}")
print(f"Protein samples: {len(samples_protein)}")

# Find common samples across all omics
common_samples = list(samples_expression.intersection(samples_methylation, samples_cnv, samples_clinical, samples_protein))
print(f"Common samples across all omics: {len(common_samples)}")

# Filter data to keep only common samples
expression_data = expression_data[common_samples]
methylation_data = methylation_data[common_samples]
copy_number_data = copy_number_data[common_samples]
protein_data = protein_data[common_samples]
phenotype_data = phenotype_data.loc[common_samples]



Checking for missing values in phenotype data:
days_to_birth.demographic                                      1
year_of_birth.demographic                                      4
days_to_death.demographic                                    170
year_of_death.demographic                                    195
entity_submitter_id.annotations                              254
notes.annotations                                            254
submitter_id.annotations                                     254
classification.annotations                                   254
entity_id.annotations                                        254
created_datetime.annotations                                 254
annotation_id.annotations                                    254
entity_type.annotations                                      254
updated_datetime.annotations                                 254
case_id.annotations                                          254
state.annotations                          

# Data preprocessing

### Check Null value

In [ ]:
#Check for null values of expression data
print("Checking for null values in expression data:")
null_expression = expression_data.isnull().sum().sum()
if null_expression > 0:
    print(f"Expression data contains {null_expression} null values.")
else:   
    print("No null values found in expression data.")
print()


# Check for null values in methylation data
print("Checking for null values in methylation data:")
null_methylation = methylation_data.isnull().sum().sum()
if null_methylation > 0:
    print(f"Methylation data contains {null_methylation} null values.")
else:
    print("No null values found in methylation data.")
print()


# Check for null values in copy number data
print("Checking for null values in copy number data:")
null_copy_number = copy_number_data.isnull().sum().sum()
if null_copy_number > 0:
    print(f"Copy number data contains {null_copy_number} null values.")
else:
    print("No null values found in copy number data.")
max_value = copy_number_data.max().max()
min_value = copy_number_data.min().min()

print(f"Maximum CNV value: {max_value}")
print(f"Minimum CNV value: {min_value}")
print()


# Check for null values in protein data
print("Checking for null values in protein data:")
null_protein = protein_data.isnull().sum().sum()
if null_protein > 0:
    print(f"Protein data contains {null_protein} null values.")
else:
    print("No null values found in protein data.")
print()


# Check for null values in phenotype data
print("Checking for null values in phenotype data:")
null_phenotype = phenotype_data.isnull().sum().sum()
if null_phenotype > 0:
    print(f"Phenotype data contains {null_phenotype} null values.")
else:
    print("No null values found in phenotype data.")



Checking for null values in expression data:
No null values found in expression data.

Checking for null values in methylation data:
Methylation data contains 16814049 null values.

Checking for null values in copy number data:
Copy number data contains 855388 null values.
Maximum CNV value: 7.0
Minimum CNV value: 0.0

Checking for null values in protein data:
Protein data contains 6300 null values.

Checking for null values in phenotype data:
Phenotype data contains 3389 null values.


### Process data

In [ ]:
# expression data preprocessing -------------------------------------------------->

# Log2 transform
expression_data_log = np.log2(expression_data + 1)

# Z-score standardization
scaler_expr = StandardScaler()
expression_data_scaled = pd.DataFrame(
    scaler_expr.fit_transform(expression_data_log.T).T,
    index=expression_data_log.index,
    columns=expression_data_log.columns
)

from sklearn.feature_selection import VarianceThreshold


# Methylation data preprocessing -------------------------------------------------->

# Droping probes with more than 10% missing values 
methylation_data = methylation_data.dropna(thresh=0.9 * methylation_data.shape[1], axis=0)

# fill na with probe wise median
methylation_data = methylation_data.apply(lambda x: x.fillna(x.median()), axis=1)


# Remove low-variance methylation probes
selector = VarianceThreshold(threshold=0.01)
methylation_filtered = pd.DataFrame(
    selector.fit_transform(methylation_data.T).T,
    index=methylation_data.index[selector.get_support()],
    columns=methylation_data.columns
)

# Z-score standardization
scaler_meth = StandardScaler()
methylation_scaled = pd.DataFrame(
    scaler_meth.fit_transform(methylation_filtered.T).T,
    index=methylation_filtered.index,
    columns=methylation_filtered.columns
)


# Copy number data preprocessing -------------------------------------------------->


# Drop genes with >20% missing values
gene_missing_threshold = 0.2
copy_number_data_filtered = copy_number_data.loc[
    copy_number_data.isnull().mean(axis=1) < gene_missing_threshold
]

# filling null values with gene-wise median
copy_number_imputed = copy_number_data_filtered.apply(
    lambda row: row.fillna(row.median()), axis=1
)

# Standardize across samples
scaler_cnv = StandardScaler()
copy_number_scaled = pd.DataFrame(
    scaler_cnv.fit_transform(copy_number_imputed.T).T,
    index=copy_number_imputed.index,
    columns=copy_number_imputed.columns
)


# Protein data preprocessing ------------------------------------------------------>

# Drop proteins with >20% missing values
protein_missing_threshold = 0.3
protein_data_filtered = protein_data.loc[
    protein_data.isnull().mean(axis=1) < protein_missing_threshold
]
# filling null values with protein-wise median
protein_imputed = protein_data_filtered.apply(
    lambda row: row.fillna(row.median()), axis=1
)

# Standardize across samples
scaler_protein = StandardScaler()
protein_scaled = pd.DataFrame(
    scaler_protein.fit_transform(protein_imputed.T).T,
    index=protein_imputed.index,
    columns=protein_imputed.columns
)

In [ ]:
# Phenotype data preprocessing -------------------------------------------------->

subtype_column = 'primary_diagnosis.diagnoses'
print(f"Using '{subtype_column}' as the subtype column")
print(f"Subtype distribution:\n{phenotype_data[subtype_column].value_counts()}")
print()

# Checking missing values in the subtype column
missing_subtypes = phenotype_data[subtype_column].isnull().sum()
print(f"Missing values in subtype column: {missing_subtypes}")

if missing_subtypes > 0:
    phenotype_data_clean = phenotype_data.dropna(subset=[subtype_column])
    print(f"Removed {missing_subtypes} samples with missing subtypes")
    print(f"Remaining samples: {len(phenotype_data_clean)}")
else:
    phenotype_data_clean = phenotype_data.copy()
    print("No missing subtypes found")

Using 'primary_diagnosis.diagnoses' as the subtype column
Subtype distribution:
primary_diagnosis.diagnoses
Leiomyosarcoma, NOS                        76
Dedifferentiated liposarcoma               49
Undifferentiated sarcoma                   32
Fibromyxosarcoma                           20
Malignant fibrous histiocytoma             11
Malignant peripheral nerve sheath tumor     9
Synovial sarcoma, spindle cell              4
Giant cell sarcoma                          3
Pleomorphic liposarcoma                     2
Myxoid leiomyosarcoma                       1
Liposarcoma, well differentiated            1
Synovial sarcoma, NOS                       1
Synovial sarcoma, biphasic                  1
Name: count, dtype: int64

Missing values in subtype column: 0
No missing subtypes found


### Sample matching

In [ ]:
# Extract subtypes for the common samples
subtypes = phenotype_data_clean[subtype_column]

# Encode subtypes as numeric labels
label_encoder = LabelEncoder()
subtype_encoded = label_encoder.fit_transform(subtypes)

# Create mapping to encode subtype classes
subtype_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print(f"\nSubtype encoding mapping:")
for subtype, encoded in subtype_mapping.items():
    print(f"  {encoded}: {subtype}")

# Converted to pandas Series to easily handle the index
subtype_encoded = pd.Series(subtype_encoded, index=subtypes.index, name='subtype_encoded')

print(f"\nEncoded subtype distribution:")
print(subtype_encoded.value_counts().sort_index())

# Update common samples with available subtypes
if missing_subtypes > 0:
    valid_samples = list(set(common_samples).intersection(set(phenotype_data_clean.index)))
    print(f"\nUpdating common samples from {len(common_samples)} to {len(valid_samples)} (removed samples with missing subtypes)")

    expression_data_scaled = expression_data_scaled[valid_samples]
    methylation_scaled = methylation_scaled[valid_samples]
    copy_number_scaled = copy_number_scaled[valid_samples]
    protein_scaled = protein_scaled[valid_samples]
    subtype_encoded = subtype_encoded.loc[valid_samples]
    
    common_samples = valid_samples
print()
print(f"Final data shapes after phenotype preprocessing:")
print(f"  Expression: {expression_data_scaled.shape}")
print(f"  Methylation: {methylation_scaled.shape}")
print(f"  Copy number: {copy_number_scaled.shape}")
print(f"  Protein: {protein_scaled.shape}")
print(f"  Phenotype: {phenotype_data_clean.shape}")
print(f"  Subtypes: {len(subtype_encoded)}")
print(f"  Common samples: {len(common_samples)}")



Subtype encoding mapping:
  0: Dedifferentiated liposarcoma
  1: Fibromyxosarcoma
  2: Giant cell sarcoma
  3: Leiomyosarcoma, NOS
  4: Liposarcoma, well differentiated
  5: Malignant fibrous histiocytoma
  6: Malignant peripheral nerve sheath tumor
  7: Myxoid leiomyosarcoma
  8: Pleomorphic liposarcoma
  9: Synovial sarcoma, NOS
  10: Synovial sarcoma, biphasic
  11: Synovial sarcoma, spindle cell
  12: Undifferentiated sarcoma

Encoded subtype distribution:
subtype_encoded
0     49
1     20
2      3
3     76
4      1
5     11
6      9
7      1
8      2
9      1
10     1
11     4
12    32
Name: count, dtype: int64

Final data shapes after phenotype preprocessing:
  Expression: (60660, 210)
  Methylation: (212863, 210)
  Copy number: (56646, 210)
  Protein: (457, 210)
  Phenotype: (210, 78)
  Subtypes: 210
  Common samples: 210


### Save preprocessed data

In [ ]:
# Save processed data
# expression_data_scaled.to_csv("processed_expression.csv")
# methylation_scaled.to_csv("processed_methylation.csv")
# copy_number_scaled.to_csv("processed_cnv.csv")
# protein_scaled.to_csv("processed_protein.csv")
# subtype_encoded.to_csv("subtype_labels.csv")

# save subtype mapping
# subtype_mapping_df = pd.DataFrame(list(subtype_mapping.items()), columns=['subtype_name', 'encoded_label'])
# subtype_mapping_df.to_csv("subtype_mapping_with_protein.csv", index=False)

# print(f"\nFinal data shapes:")
# print(f"Expression: {expression_data_scaled.shape}")
# print(f"Methylation: {methylation_scaled.shape}")
# print(f"Copy number: {copy_number_scaled.shape}")
# print(f"Protein: {protein_scaled.shape}")
# print(f"Subtypes: {len(subtype_encoded)}")
# print(f"Subtype classes: {len(label_encoder.classes_)}")
# print("\nProcessed data saved to CSV files:")
# print("- processed_expression.csv")
# print("- processed_methylation.csv") 
# print("- processed_protein.csv")
# print("- processed_cnv.csv")
# print("- subtype_labels.csv")
# print("- subtype_mapping.csv")

In [ ]:
print("Expression:", expression_data_scaled.shape)
print("Methylation:", methylation_scaled.shape)
print("Copy Number:", copy_number_scaled.shape)
print("Protein:", protein_scaled.shape)
print("Phenotype:", phenotype_data_clean.shape)
print("Subtype labels:", subtype_encoded.shape)


Expression: (60660, 210)
Methylation: (212863, 210)
Copy Number: (56646, 210)
Protein: (457, 210)
Phenotype: (210, 78)
Subtype labels: (210,)


In [ ]:
print(expression_data_scaled.iloc[:5, :5])  # First 5 rows/columns
print(methylation_scaled.iloc[:5, :5])
print(copy_number_scaled.iloc[:5, :5])
print(protein_scaled.iloc[:5, :5])
print(subtype_encoded.head())


                    TCGA-MB-A5YA-01A  TCGA-X6-A7WD-01A  TCGA-DX-AB3C-01A  \
Ensembl_ID                                                                 
ENSG00000000003.15         -0.991605         -0.467354          0.135911   
ENSG00000000005.6           0.279301         -0.447155         -0.502103   
ENSG00000000419.13         -0.409478          0.048320         -0.923084   
ENSG00000000457.14         -0.598295          0.514173         -0.287709   
ENSG00000000460.17         -0.968116          0.617331         -0.827791   

                    TCGA-DX-A3M1-01A  TCGA-DX-A48O-01A  
Ensembl_ID                                              
ENSG00000000003.15          0.606595         -1.274259  
ENSG00000000005.6           0.440239          0.848149  
ENSG00000000419.13          0.291095         -0.470558  
ENSG00000000457.14         -0.434324          0.435434  
ENSG00000000460.17          0.313320          0.085932  
                       TCGA-MB-A5YA-01A  TCGA-X6-A7WD-01A  TCGA-DX-A

# Reduce Dimension

### Autoencoder

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


class Autoencoder(nn.Module):
    def __init__(self, input_dim, hidden_dim=256, latent_dim=64):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(inplace=True),
            nn.Linear(hidden_dim, latent_dim),
            nn.ReLU(inplace=True)
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim),
            nn.ReLU(inplace=True),
            nn.Linear(hidden_dim, input_dim),
        )

    def forward(self, x):
        z = self.encoder(x)  # Bottleneck representation
        x_hat = self.decoder(z)  # Decoder representation
        return x_hat, z
    
    def encode(self, x):
        """Extract bottleneck representation"""
        return self.encoder(x)
    
    def decode(self, z):
        """Extract decoder representation"""
        return self.decoder(z)

def extract_representations(model, tensor_data):
    """Extract bottleneck and decoder representations from trained autoencoder"""
    model.eval()
    with torch.no_grad():
        # Get bottleneck (latent) representation
        bottleneck = model.encode(tensor_data)
        # Get decoder representation (reconstructed features)
        decoder_repr = model.decode(bottleneck)
    return bottleneck.cpu().numpy(), decoder_repr.cpu().numpy()

# Initialize autoencoders for each omics type (without training)
omics_scaled = {
    'expr': expression_data_scaled,
    'meth': methylation_scaled,
    'cnv': copy_number_scaled,
    'prot': protein_scaled
}

# Store autoencoder models (to be trained later)
autoencoder_models = {}
bottleneck_embeds = {}
decoder_embeds = {}

for name, df in omics_scaled.items():
    print(f">> Initializing autoencoder for {name} modality")

    X = torch.tensor(df.values.T, dtype=torch.float32, device=device)  # (samples, features) -- Transpose to (features, samples)
    input_dim = X.shape[1]
    sample_ids = df.columns
    
    print(f"Data shape: {df.shape} -> After transpose: {X.shape}")
    print(f"Input dimension (features): {input_dim}")
    
    # Initialize autoencoder
    model = Autoencoder(
        input_dim=input_dim, 
        hidden_dim=128,
        latent_dim=64
    ).to(device)
    autoencoder_models[name] = model

    bottleneck, decoder_repr = extract_representations(model, X)
    
    print(f"Index colomn shape: {df.columns.shape}")
    bottleneck_embeds[name] = pd.DataFrame(
        bottleneck,
        index=sample_ids,  # Sample IDs
        columns=[f"{name}_bottleneck_{i}" for i in range(bottleneck.shape[1])]
    )
    
    print(f"Index colomn shape: {df.columns.shape}")
    decoder_embeds[name] = pd.DataFrame(
        decoder_repr,
        index=sample_ids,  # Sample IDs
        columns=[f"{name}_decoder_{i}" for i in range(decoder_repr.shape[1])]
    )
    
    print(f"   Bottleneck shape: {bottleneck_embeds[name].shape}")
    print(f"   Decoder shape: {decoder_embeds[name].shape}")
    print()

>> Initializing autoencoder for expr modality
Data shape: (60660, 210) -> After transpose: torch.Size([210, 60660])
Input dimension (features): 60660
Index colomn shape: (210,)
Index colomn shape: (210,)
   Bottleneck shape: (210, 64)
   Decoder shape: (210, 60660)

>> Initializing autoencoder for meth modality
Data shape: (212863, 210) -> After transpose: torch.Size([210, 212863])
Input dimension (features): 212863
Index colomn shape: (210,)
Index colomn shape: (210,)
   Bottleneck shape: (210, 64)
   Decoder shape: (210, 212863)

>> Initializing autoencoder for cnv modality
Data shape: (56646, 210) -> After transpose: torch.Size([210, 56646])
Input dimension (features): 56646
Index colomn shape: (210,)
Index colomn shape: (210,)
   Bottleneck shape: (210, 64)
   Decoder shape: (210, 56646)

>> Initializing autoencoder for prot modality
Data shape: (457, 210) -> After transpose: torch.Size([210, 457])
Input dimension (features): 457
Index colomn shape: (210,)
Index colomn shape: (210,

In [ ]:
# # Create fusion models
# print("\n" + "="*50)
# print("Creating Fusion Models")
# print("="*50)

# # Fusion Model 1: Using bottleneck representations
# bottleneck_fusion = pd.concat(
#     [bottleneck_embeds['expr'], bottleneck_embeds['meth'], 
#      bottleneck_embeds['cnv'], bottleneck_embeds['prot']],
#     axis=1
# )

# # Fusion Model 2: Using decoder representations
# decoder_fusion = pd.concat(
#     [decoder_embeds['expr'], decoder_embeds['meth'], 
#      decoder_embeds['cnv'], decoder_embeds['prot']],
#     axis=1
# )

# print(f"Bottleneck fusion shape: {bottleneck_fusion.shape}")
# print(f"Decoder fusion shape: {decoder_fusion.shape}")

# print("\nFusion models created successfully!")
# print("Note: These use untrained autoencoders. Train the models first, then re-extract representations.")

# # Display sample data
# print(f"\nBottleneck fusion sample:")
# print(bottleneck_fusion.head())

# print(f"\nDecoder fusion sample:")
# print(decoder_fusion.head())